In [1]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader, 
    StorageContext,
    load_index_from_storage,
    Document,
    PromptTemplate
)
load_dotenv()

True

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model

In [3]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [5]:
import os
PERSIST_DIR = "lyrics_store_shakira"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [6]:
query_engine = index.as_query_engine()

In [7]:
qa_template_str = """
    You are an expert in Bolivian Folk music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)

In [8]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [12]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

description = """
A set of lyrics for songs from the artist called shakira. 
Use plain text question as input to the tool. 
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool.
"""
tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="shakira_songs_lyrics",
            description=description,
            return_direct=False
        )
    )
]
agent = OpenAIAgent.from_tools(tools=tools, verbose=True)

In [13]:
response = agent.chat("que canciones de sharira habla de animales?")

Added user message to memory: que canciones de sharira habla de animales?
=== Calling Function ===
Calling function: shakira_songs_lyrics with args: {"input":"songs about animals"}
Got output: - La canción "Animal City" de Shakira es una canción que aborda el tema de la fama y la presión social en un entorno competitivo y a menudo despiadado. Algunas de las letras de la canción incluyen frases como "It's an animal city, It's a cannibal world" y "Some are ready to fight you", lo que sugiere una atmósfera intensa y competitiva.

- En la canción, Shakira reflexiona sobre cómo la fama puede cambiar la percepción de las personas hacia uno, mencionando frases como "Your family got bigger when they thought you were rich" y "And once you have become a star, you got no right to bitch".

- La canción también destaca la importancia de mantenerse fiel a uno mismo en medio de la presión externa, con frases como "But you're the real deal and 'real' is your middle name" y "With you I feel safe, there

In [11]:
response.response

'Lo siento, pero en el contexto proporcionado no hay información sobre canciones de Kjarkas que hablen de animales. La información disponible se centra en canciones de Shakira.'

In [14]:
response = agent.chat("que hay de loba?")

Added user message to memory: que hay de loba?
=== Calling Function ===
Calling function: shakira_songs_lyrics with args: {"input":"lyrics of the song loba"}
Got output: - La canción "Loba" de Shakira tiene los siguientes versos:
  - "Loba, sigilosa al pasar, sigilosa al pasar"
  - "Una loba en el armario, tiene ganas de salir, deja que se coma el barrio antes de irte a dormir"
  - "Mis aullidos son el llamado, yo quiero un lobo domesticado"
  - "La luna llena como una fruta, no da consejos ni los escucha"
  - "Deja que se coma el barrio antes de irte a dormir"

- El tono/sentimiento de la canción "Loba" es desafiante y liberador, habla de una mujer fuerte y segura de sí misma.

